In [2]:
from pyomo.environ import *
# Pyomo core library:
# - ConcreteModel → container for the optimization problem
# - Param → fixed input numbers (scenario inputs)
# - Var → decision variables (what solver decides)
# - Constraint → business / physical rules
# - Objective → what we optimize (min / max)
# - SolverFactory → connects Pyomo to a solver (HiGHS here)


def main():
    # ============================================================
    # 1️⃣ MODEL CONTAINER
    # ============================================================
    # This is an EMPTY box at the beginning.
    # We will slowly put:
    # - data (parameters)
    # - decisions (variables)
    # - rules (constraints)
    # - goal (objective)
    #
    # Think of it as: "digital twin of our supply chain"
    m = ConcreteModel()

    # ============================================================
    # 2️⃣ PARAMETERS (Scenario 1: Demand +20%)
    # ============================================================
    # PARAMETERS are numbers GIVEN to the model.
    # The solver CANNOT change them.
    #
    # Planner thinking:
    # "What is fixed in this scenario?"

    m.demand = Param(
        initialize=120
    )
    # Customer demand = 120 apples
    # This is the SCENARIO CHANGE (+20%)
    # Base was 100 → now customers want more

    m.capacity = Param(
        initialize=100
    )
    # Truck capacity = 100 apples
    # Physical limit: cannot ship more than this

    m.cost_per_apple = Param(
        initialize=5
    )
    # Transportation cost per apple
    # Represents fuel, driver, wear & tear

    m.penalty_unmet = Param(
        initialize=20
    )
    # Penalty cost for EACH unmet apple
    # This is NOT transport cost
    # This represents:
    # - lost sales
    # - lost customers
    # - emergency sourcing
    # - planner pain 😄

    # ============================================================
    # 3️⃣ DECISION VARIABLES
    # ============================================================
    # VARIABLES are decided by the solver.
    # The solver asks:
    # "What values minimize total cost?"

    m.shipped = Var(
        domain=NonNegativeReals
    )
    # How many apples do we actually ship?
    # Must be >= 0
    # Solver will push this UP until capacity limit

    m.unmet = Var(
        domain=NonNegativeReals
    )
    # How many apples we FAIL to deliver
    # This variable makes the model realistic
    # Instead of pretending demand is always met

    # ============================================================
    # 4️⃣ CONSTRAINTS (Rules of the world)
    # ============================================================

    # ----------------------------
    # Capacity constraint
    # ----------------------------
    m.capacity_constraint = Constraint(
        expr=m.shipped <= m.capacity
    )
    # Meaning in plain English:
    # "You cannot load more apples than the truck can carry"
    #
    # If demand > capacity → solver must create unmet demand

    # ----------------------------
    # Demand balance constraint
    # ----------------------------
    m.demand_constraint = Constraint(
        expr=m.shipped + m.unmet == m.demand
    )
    # VERY IMPORTANT constraint
    #
    # It forces accounting correctness:
    # Every apple demanded must be:
    # - either shipped
    # - or unmet
    #
    # No disappearing apples
    # No magic fulfillment

    # ============================================================
    # 5️⃣ OBJECTIVE FUNCTION (Business goal)
    # ============================================================
    m.obj = Objective(
        expr=
            m.shipped * m.cost_per_apple +
            m.unmet * m.penalty_unmet,
        sense=minimize
    )

    # Objective explanation:
    #
    # Total Cost =
    #   transport cost (per apple shipped)
    # + penalty cost (per apple unmet)
    #
    # Solver trade-off logic:
    # - Shipping apples costs money
    # - NOT shipping apples costs EVEN MORE money
    #
    # This forces the solver to:
    # - ship as much as capacity allows
    # - accept unmet demand only when unavoidable

    # ============================================================
    # 6️⃣ SOLVE THE MODEL
    # ============================================================
    solver = SolverFactory("highs")
    # HiGHS is a fast LP/MIP solver
    # Perfect for supply chain linear models

    solver.solve(m)
    # Solver now:
    # - reads constraints
    # - searches feasible region
    # - finds minimum-cost solution

    # ============================================================
    # 7️⃣ RESULTS & KPIs (Planner language)
    # ============================================================

    shipped = value(m.shipped)
    # Optimal shipped quantity

    unmet = value(m.unmet)
    # Optimal unmet demand

    demand = value(m.demand)
    # Scenario demand (fixed input)

    service_level = shipped / demand
    # KPI: Service Level
    # % of demand successfully fulfilled

    print("=== SCENARIO 1: DEMAND +20% ===")
    print(f"Demand           : {demand}")
    print(f"Shipped          : {shipped}")
    print(f"Unmet Demand     : {unmet}")
    print(f"Service Level    : {service_level:.2%}")
    print(f"Total Cost       : {value(m.obj)}")
    # These prints are what planners care about,
    # not variable names or solver logs


# ============================================================
# 8️⃣ PYTHON ENTRY POINT (IMPORTANT)
# ============================================================
if __name__ == "__main__":
    main()

# Meaning:
# - If this file is run directly → execute main()
# - If this file is imported elsewhere → do NOT auto-run
#
# This is standard Python engineering practice


=== SCENARIO 1: DEMAND +20% ===
Demand           : 120
Shipped          : 100.0
Unmet Demand     : 20.0
Service Level    : 83.33%
Total Cost       : 900.0
